# P-ai AI/ML Workshop: Session 4

Welcome to P-ai's third session of the AI/ML workshop series! Today we'll learn about
- Deep learning
    - Intuition behind neural nets
    - How to build and train a neural net with Tensorflow and Keras
    - Types of neural nets
- Solving a real-world classification problem with a neural net

<img src="https://images.squarespace-cdn.com/content/5d5aca05ce74150001a5af3e/1580018583262-NKE94RECI46GRULKS152/Screen+Shot+2019-12-05+at+11.18.53+AM.png?content-type=image%2Fpng" width="200px">

## 1. Feed-forward Neural Nets

### A simple example

Let's say we have a text, and we want to predict whether the language of the text is English, Spanish, or German. One simple way to attempt this would be to calculate the frequency of each letter in the alphabet, and use those frequencies to predict the language. For this example, we'll only use the 26 standard English letters, although in practice, that would be a very dumb decision since the presence of letters like `ñ` or `ü` would pretty much immediately give away the answer.

So, our features would look like:
- $x_1$ frequency of the letter `a`
- $x_2$ frequency of the letter `b`  
...
- $x_{26}$ frequency of the letter `z`

To build a little bit of intuition for neural nets, let's say we saw this string of text:

In [1]:
text = 'Mi abuela tiene noventa y cuatro años. Cuando era joven trabajó como enfermera en un hospital. Ahora le gustan las manualidades y hace pulseras para toda la familia. Por las mañanas sale a dar un paseo con sus amigas y por las tardes ve la televisión.'
# Remove special characters for the sake of example
text = text.replace('ñ', 'n')
text = text.replace('ó', 'o')
text

'Mi abuela tiene noventa y cuatro anos. Cuando era joven trabajo como enfermera en un hospital. Ahora le gustan las manualidades y hace pulseras para toda la familia. Por las mananas sale a dar un paseo con sus amigas y por las tardes ve la television.'

In [2]:
# Helpful for counting
from collections import Counter
# Case doesn't matter
text = text.lower()
# Punctuation doesn't matter
text = text.replace(' ','')
text = text.replace('.','')
text = text.replace(',','')
text_len = len(text)
counter = Counter(text)
# Sort entries
counts = dict(sorted(counter.items(), key=lambda item: -item[1]))
print('Letter frequencies\n------------------')
for letter in counts:
    print(f"{letter}: {counts[letter] / text_len}")

Letter frequencies
------------------
a: 0.19306930693069307
e: 0.09900990099009901
s: 0.08415841584158416
n: 0.07920792079207921
o: 0.07920792079207921
l: 0.06435643564356436
r: 0.0594059405940594
i: 0.04455445544554455
u: 0.04455445544554455
t: 0.04455445544554455
m: 0.034653465346534656
d: 0.0297029702970297
p: 0.0297029702970297
c: 0.024752475247524754
v: 0.019801980198019802
y: 0.01485148514851485
h: 0.01485148514851485
b: 0.009900990099009901
j: 0.009900990099009901
f: 0.009900990099009901
g: 0.009900990099009901


It's not exactly trivial to guess which language this is from the frequencies alone, but you might have a bit of intuition. For example, `t` is extremely common in English (the second most frequent letter, actually), so we might suspect this isn't English given it's nearly halway down the frequency list. In other words, the weight for $x_{20}$ (the frequency of the letter `t`) should be higher for English than for Spanish.

If we assume that the probability of the text being a certain language is linearly related to these frequencies, we could frame this problem like a logistic regression problem:

$$
w_{1e}x_1 + w_{2e}x_2 + {...} + w_{26e}x_{26} + b_e = logit_e
$$

Where $x_1$ is the frequency of letter `a` in our text, $w_{1e}$ is its respective weight (for English), and ditto for the rest of the input varluables. $b_e$ is the bias.

> What's up with the logit?

The output of a linear combination in linear regression is called a **logit**; it's related to the probability of the input data belonging to a certain class, but it's not quite a probability yet (which should be apparent because it can be any real value, whereas probabilities should be between `0` and `1`). For binary tasks, we used the `sigmoid` to get from a logit to a probability, but we'll need to do something different for multiclass classification– we'll get to that in a second!


### More on logits

$$
logit(p) = ln\left( \frac{p}{1-p} \right)
$$

Discussion question: What is the behavior of this function? Why might this form be useful for classification tasks?


Notice that we'll have something very similar for Spanish and German, but with different weights and biases.

$$
w_{1s}x_1 + w_{2s}x_2 + {...} + w_{26s}x_{26} + b_s = logit_s \\
w_{1g}x_1 + w_{2g}x_2 + {...} + w_{26g}x_{26} + b_g = logit_g
$$

Note how every feature ($x_1$ through $x_{26}$) gets 'sent' to all three outputs. Conversely, each output receives all the inputs. We can sketch our neural net like this:

<img src="nn_1.png" width="500px">

Again, this is just a visual representation of the three equations we wrote above, we haven't made any leaps of logic. An arrow coming out of a neuron means to multiply its value with a weight. All the arrows entering a neuron means to sum all those products together. With that in mind, you should be able to see that we're just representing equations with neurons and arrows.

Also, notice the dimensions. Our input layer has shape `(26,)` (vector of size 26) and the output layer has shape `(3,)` (vector of size 3).

Dimensionality is a very important aspect of neural networks. You'll see why in a bit!

In [3]:
# Illustrate the point by creating the input vector from the example above
import numpy as np
x = np.zeros(shape=(26,), dtype=np.float32)                # Create empty vector
for i, letter in enumerate('abcdefghijklmnopqrstuvwxyz'):  # Iterate through 26 letters
    if letter in counts:                                   # If the ith letter appeared in the text
        x[i] = counts[letter]           # Set x_i to that letter's count
x = np.divide(x, sum(x))                # Turn counts into frequencies
print("X_vector:\n", x)
print("Shape:", x.shape)

X_vector:
 [0.19306931 0.00990099 0.02475248 0.02970297 0.0990099  0.00990099
 0.00990099 0.01485149 0.04455446 0.00990099 0.         0.06435644
 0.03465347 0.07920792 0.07920792 0.02970297 0.         0.05940594
 0.08415841 0.04455446 0.04455446 0.01980198 0.         0.
 0.01485149 0.        ]
Shape: (26,)


Notice how we turned an input (text, in this case) into a vector. This vector would then go into the neural net during training (which is how the model gets $x_1$ through $x_{26}$).

> What about the $y$ data? Why is it a vector of size 3?

For binary classification, our $y$ data can simply be a `1` or a `0`; either an input belongs to the class, or it does not. For multiclass classification, we think of having a separate probability for the input belonging to each class. Since our task has three languages we're trying to classify between, our output vector would have size 3. For example, the text we saw above was Spanish, so the correct $y$ would be `[0, 1, 0]`, assuming each value is the probability of the text being English, Spanish, and German, respectively (we could have picked any order; we just need to stick with one). This is how we typically encode multiclass variables (one hot encoding) for both inputs and outputs.


### Activation Functions

Discussion time: Let's think about a binary classification task where we are trying to classify something as a 0 or a 1. How do we get from logits to something in the range of [0, 1]? In other words, how do we normalize?

We need something that sends 
$$
logit(p) = ln\left( \frac{p}{1-p} \right)
$$
to [0, 1]

<img src = "https://miro.medium.com/max/640/1*Xu7B5y9gp0iL5ooBj7LtWw.png">
(Sigmoid)

What if we want [-1, 1]?

<img src = "https://www.medcalc.org/manual/functions/tanh.png"> (tanh)

But wait... for our case, we have three classes. How do we accomplish that? Sigmoid only does [0, 1], tanh only does [-1, 1]

**The softmax function**: A generalized form of sigmoid. Sigmoid, but on every single logit!

<img src = "softmax.png">

$$
\sigma(z_i) = \frac{e^{z_{i}}}{\sum_{j=1}^K e^{z_{j}}} \ \ \ for\ i=1,2,\dots,
$$

z is the input vectors to the softmax, zi are the individual elements in the vector, K denotes the number of classes. So we see we can stick an infinite amount of classes in here and it'll always be normalized!

Discussion time: Activation functions sure are useful for transforming our data into classes, but what else might they be good for?

<img src = "https://www.nomidl.com/wp-content/uploads/2022/04/image-10.png" width = 400px> (relu)
<img src = "https://production-media.paperswithcode.com/methods/Screen_Shot_2020-05-25_at_3.09.45_PM.png" width = 300px> (leaky relu)

Why might leaky relu be useful?

<img src = "http://cdn.shopify.com/s/files/1/1186/5476/products/chefs-knife-black-191015-shoppage_63c5525d-e6a0-4e39-a59a-6e10ee351fd4_3311x.jpg?v=1650903908" width = 300px>
<img src = "https://www.nomidl.com/wp-content/uploads/2022/04/image-10.png" width = 400px>

### The holy grail: Non-linearity

Discussion time: Why does having a nonlinear activation function matter? Let's think about this in terms of linear algebra... Recall this equation:

$$
w_{1s}x_1 + w_{2s}x_2 + {...} + w_{26s}x_{26} + b_s = logit_s \\
w_{1g}x_1 + w_{2g}x_2 + {...} + w_{26g}x_{26} + b_g = logit_g
$$

If we want to increase the complexity of our model to work on harder tasks, we can add more layers. However, what happens if we only add linear layers?


>So to review what we have in this architecture:

This is a **neural network** with an **input layer**, an **output layer**, and **no hidden layers**. The input layer has **26 neurons** and the output layer has **3 neurons**. There are **weights** (multiply with the input) that connect each node in first layer to the second layer, and **biases** (add with the input) for each neuron in the output layer. We apply the **softmax activation function** to the last layer of the neural net to output a probability vector.

<img src="nn_2.png" width="500px">

>So... how does it learn?

The model makes a certain number of predictions and calculates a **loss** (see Workshop 3 for a more detailed explanation of this). The model then uses **gradient descent** and a process called **backpropagation** (shortened to backprop or just BP) to iteratively adjust the weights and biases to improve the model's performance (minimize loss). Then, the model makes more predictions, and the whole process repeats.

In a little more detail:
1. Initially, the model's parameters are randomly or pseudo-randomly assigned
    - Two common initializers are [He](https://faroit.com/keras-docs/2.0.0/initializers/#he_uniform) and [Glorot](https://faroit.com/keras-docs/2.0.0/initializers/#glorot_uniform)
2. The model makes a prediction (or many); that just means running an input vector through the layers until a probability vector is output
    - See **batches**, explained below
3. The output vector (the prediction) is compared to the actual $y$ vector (e.g. `[0.2, 0, 0.8] <=> [0, 0, 1]`)
4. The model uses the loss function to calculate a **loss** (think error) from the predicted and actual output vectors
5. Using the loss, the model uses **backpropagation** to *propagate the error backwards through the layers*, adjusting the weights and biases along the way to hopefully decrease the loss
6. Steps 2-6 repeat until some stop condition is reached
    - For example, you may tell the model to train on all of the input data 5 times (5 **epochs**)
    - Another method is to automatically stop training when the validation loss stops decreasing

### Hidden layers

This is all well and good, but we may suspect that the probem is more complex than pure linear combinations. This is what leads us to add **hidden layers** between our input and output layers, which allow for the model to learn more complex relationships between variables. That would look something like this:

<img src="nn_3.png" width="800px">

>Again, note the architecture:

Everything is the same as the previous example, except we now have **two hidden layers**. In our code, we would say how many neurons each of these hidden layers has. For example, the first could have 128 and the second could have 256 (powers of 2 are common). All of these layers are **densely connected**, meaning every neuron is connected to every neuron in the next layer.

### Terminology summary

- `Neuron`: A neuron really just stores a value either from the input vector or preceding neuron(s), and then passes that value to the next neuron(s)
- `Layer`: A layer is a collection of neurons at the same "depth" of the neural network. This is usually easier seen than explained
    - `Input layer`: The first layer in a neural net; it takes in an input vector
    - `Hidden layer`: A layer between the input and output layer
    - `Output layer`: The last layer in a neural net; it produces the output of the net
- `Dense layer`: A layer in which every neuron is connected to every neuron in the previous layer
- `Weight`: A weight assigned to a pair of neurons in different layers. For example, if there are two densely connected layers, every neuron in the first layer is connected to a neuron in the second layer with a weight. The value of the first neuron gets multiplied by the weight and send to the second neuron.
- `Bias`: A value that gets added to every neuron in a layer; these values do not depend on the input
- `Trainable parameters`: The parameters of the model that are tuned during training; often, this means the weights and biases. Sometimes, people say "parameters" to mean the trainable parameters specifically
- `Hyperparameters`: Parameters of the model that aren't tuned through training, like the number of layers, number of neurons in each layer, learning rate, etc.
- `Loss`: A measure of "badness" comparing the model predictions to the actual outputs
- `Loss function`: A function that compares the predicted and actual outputs and produces a loss. There are a few loss functions that are very commonly used for certain problems. These are:
    - [Binary crossentropy](https://keras.io/api/losses/probabilistic_losses/#binary_crossentropy-function) for binary classification
    - [Categorical crossentropy](https://keras.io/api/losses/probabilistic_losses/#categorical_crossentropy-function) for multiclass classification
    - [Mean Squared Error](https://keras.io/api/losses/regression_losses/#mean_squared_error-function) for regression
    - Check out [https://keras.io/api/losses/](https://keras.io/api/losses/) for a complete list of Keras's loss functions!
- `Gradient descent`: The process by which neural nets (and other models) make predictions to estimate the loss gradient and then iteratively tweak parameters to decrease the loss
- `Learning rate`: This is like the step size of the gradient descent. A lower learning rate is generally more stable but requires more data to learn more, whereas a higher learning rate can potentially get to the local minimum faster, but might also "overshoot" it many times in the process.
- `Optimizer`: The method of gradient descent. Some common ones include [Adam](https://keras.io/api/optimizers/adam/) and [SGD](https://keras.io/api/optimizers/sgd/).
- `Backpropagation`: The process that describes *how* to update the weights and biases across the (potentially many) layers of a neural net. It's called *back* propagation because the error is propagated *backwards* through the layers
- `Epoch`: One complete "run through" of the training data. That is, if the model has been trained for 10 epochs, then it has seen every example in the training data 10 times
- `Batch`: A collection of examples that the model makes predictions on before backpropagating
    - `Batch size`: The size of each batch
    - `Stochastic gradient descent`: Essentially a batch size of 1; backprop after each prediction
    - `Minibatch gradient descent`: Split the training data into batches, backprop after each batch
    - `Batch gradient descent`: Treat the entire dataset as a batch; backprop after each epoch
    
You can read more about stochastic / minibatch / batch GD [here](https://sweta-nit.medium.com/batch-mini-batch-and-stochastic-gradient-descent-e9bc4cacd461)!

### Backpropagation

Recall our talk about gradient descent and error last time. That was for a single layer. However, if we have multiple layers, how do we understand the error that is generated by each layer?

Let's first look at what happens in forward propagation:

<img src = "https://miro.medium.com/max/720/1*51X_xj8p-jO8-plMfsyajg.png">

z defines our neuron value, a denotes the value upon applying the activation function

In the end, we compute some loss that determines how far off our prediction is from the actual:

$$ J = loss(s_{actual}, s_{pred})$$

How can we figure out how each layer contributes to this loss?

**We gotta go backwards**

Let's imagine this simplified architecture, a single neuron model with multiple layers:

<img src = "simplesingle.png" width = 800px>

For a single layer, we can write:
$$\frac{\partial J}{\partial w_k} =
      \frac{\partial J}{\partial z_{k}} \frac{\partial z_{k}}{\partial w_{{k}}}$$

where k is the layer you are on

Now that we have the single layer, how do we capture the error due to the weight w(k-2) in layer k - 2?
$$\frac{\partial J}{\partial w_{k-2}} =
      \frac{\partial J}{\partial z_{k}} \frac{\partial z_{k}}{\partial w_{{k-1}}} \frac{\partial w_{{k-1}}}{\partial z_{k-1}} \frac{\partial z_{k-1}}{\partial w_{{k-2}}}$$

Well... How do we get all the way back to the beginning?